# Advanced Types

## Composite Types
Composite types are user-defined objects that store structured data. They are defined in Julia with the construct struct.
***
Imagine that you want to read, store, and manipulate the information from the survey, perhaps with thousands of observations. You soon realize that you have plenty of data that comes in a non-conventional form: part of it is in terms of integers, part in terms of a string, part in terms of a floating, etc. In some datasets, the information may even contain complex Unicode characters, images, maps, etc. You could construe arrays to store that information (Julia allows for arrays with multiple types), but, after some time you will find that the approach generates complex code.

**A much simpler strategy is to design your own type.**

In [1]:
struct MicroSurveyObservation
    id::Int
    year::Int
    quarter::Int
    region::String
    ageHouseholdHead::Int
    familySize::Int
    numberChildrenunder18::Int
    consumption::Float64
end

**Note**: In the above example, we have annotated all fields with the operator :: . 


This declaration is not necessary as a field without explicit declaration will default to any.


In [3]:
# alternative constructor of MicroSurveyObservation

struct MicroSurveyObservation_implicit
    id
    # other fields here
end

#### Performing Basic Operations

In [4]:
#Creating an instance for the fiels is pretty straightforward
household1 = MicroSurveyObservation(12,2017,3,"angushire",23,2,0,345.34)

MicroSurveyObservation(12, 2017, 3, "angushire", 23, 2, 0, 345.34)

In [10]:
#You can check the names of all the fields with
fieldnames(MicroSurveyObservation)

#To access to any of these fields, you only need to use a . after the name of the variable followed by the field:
household1.familySize

2

**Note** Objects that are created with structs are immutable


Ex: household1.id = 31 --> Will Generate An Error Message

***
Creating a different variable for each observation in our survey is not very efficient. Imagine that we have n observations. Then, we can define an abstract array n × 1 and populate it with repeated applications of the constructor:

In [6]:
household = Array{Any}(undef,2,1)
household[1] = MicroSurveyObservation(12,2017,3,"angushire", 23, 2,0,345.34)
household[2] = MicroSurveyObservation(13,2015,2,"Wolpex", 35, 5,2,645.34)

MicroSurveyObservation(13, 2015, 2, "Wolpex", 35, 5, 2, 645.34)

Even more efficiently, you can build a loop that reads data from a file and builds each element of the array:

In [18]:
household = Array{Any}(undef,10,1)
for i in 1:10
    household[i] = MicroSurveyObservation(12,2017,3,"angushire", 23, 2,0,345.34)
end

If you have experience with other object-oriented languages you would have recognized that a composite type is similar to a class in C++, Python, R, or Matlab or a structure in C/C++ or Matlab22. At the same time, you might miss the definition of methods in the class. In comparison with object-oriented languages, in Julia, functions are not tied with objects.
***
This is a second key difference of multiple dispatch with respect to operator overloading: in Julia you will take an existing function and add a new method to it to deal with a concrete composite type or create a new function with its specific method if you want to have a completely new operation.

In [19]:
import Base: + # importing + from base package

# definition of sum function for MicroSurveyObservation composite types 
+(x::MicroSurveyObservation,y::MicroSurveyObservation) = x.consumption + y.consumption

#= This function extends the sum operator + to instances of MicroSurveyObservation .
#Basically we are taking the function + in julia and we are adding to it a new method to deal with our composite type
=#

# an example of how to apply the sum
household[1]+household[2] #return 991

equivConsumption(x::MicroSurveyObservation) = x.consumption/sqrt(x.familySize)
equivConsumption(household[1])

244.1922558149623

Note the flexibility of working with composite types in this way: if you decide to define a new household equivalence scale you only need to change the function equivConsumption() without worrying about the data structure itself. In comparison, with classes in C++ or Matlab, you would need to change the definition of the class itself by introducing a new operator.
***
One could als declare a type that holds a pair of real numbers, subject to the constraint that the first number is not greater than the second one. One could declare it like this:

In [20]:
struct OrderedPair
    x::Real
    y::Real
    OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

In [21]:
OrderedPair(1, 2)

OrderedPair(1, 2)

In [22]:
OrderedPair(2,1) #Generates an Error

LoadError: out of order

## Mutable Composite Types
***
If a composite type is declared with mutable struct instead of struct, then instances of it can be modified:

In [23]:
mutable struct MicroSurveyObservationMut
    id::Int
    #... 
end

In [25]:
number = MicroSurveyObservationMut(1)

MicroSurveyObservationMut(1)

In [27]:
#now the instance can be modified
number.id = 5

5

## Parametric Composite Types (TBD)

In [28]:
#Type parameters are introduced immediately after the type name, surrounded by curly braces
struct Point{T}
           x::T
           y::T
end

This declaration defines a new parametric type, Point{T}, holding two "coordinates" of type T.
 - What, one may ask, is T? Well, that's precisely the point of parametric types: it can be any type at all.
 
 
Thus, this single declaration actually declares an unlimited number of types: Point{Float64}, Point{AbstractString}, Point{Int64}, etc. Each of these is now a usable concrete type:

In [29]:
Point{Float64} #is a point whose coordinates are 64-bit floating-point values

Point{AbstractString} #is a "point" whose "coordinates" are string objects

Point{AbstractString}

**There are two default ways of creating new composite objects:**
1. the type parameters are explicitly given
2. the type parameters are implied by the arguments to the object constructor.

In [30]:
#1 --> explicit
p = Point{Float64}(1.0, 2.0)
typeof(p)

Point{Float64}

In [31]:
#2 --> Implicit. NOTE: the two arguments must have the same type
p1 = Point(1.0,2.0)
typeof(p1)

Point{Float64}


**Note**: Methods or functions can be broadcasted to subtypes (or at least that's what I understood). 

Ex: Since Point{Float64} is not a subtype of Point{Real}, the following method can't be applied to arguments of type Point{Float64}:

In [32]:
function norm(p::Point{Real})
    sqrt(p.x^2 + p.y^2)
end

#the above method can only be applied to Point{Real}

function norm(p::Point{<:Real})
    sqrt(p.x^2 + p.y^2)
end

#the above method can only be applied to all Point{T} where T is a subtype of Real

#The following are equivalent definitions of the previous function

function norm(p::Point{T} where T<:Real)
    sqrt(p.x^2 + p.y^2)
end

function norm(p::Point{T}) where T<:Real
    sqrt(p.x^2 + p.y^2)
end

norm (generic function with 2 methods)

## Type Union
***
A type union is a special abstract type which includes as objects all instances of any of its argument types.

In [33]:
IntOrString = Union{Int,AbstractString}

Union{Int64, AbstractString}

In [34]:
1 :: IntOrString
"Hello" :: IntOrString

"Hello"

In [38]:
1.0 :: IntOrString #This raises an error because IntOrString can either be an Int or an AbsrtactString

LoadError: TypeError: in typeassert, expected Union{Int64, AbstractString}, got a value of type Float64

A particularly useful case of a Union type is Union{T, Nothing}. Declaring a function argument or a field as Union{T, Nothing} allows setting it either to a value of type T, or to nothing to indicate that there is no value.

**Note: there are many other types that I will not Discuss. If you're interested, you can check the documentation.**